In [5]:
!python -m pip install -U pip

Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/fe/ef/60d7ba03b5c442309ef42e7d69959f73aacccd0d86008362a681c4698e83/pip-21.0.1-py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [10]:
print("ali")

ali


In [15]:
!pip install pandas
!pip install numpy
!pip install python-binance
!pip install python-dotenv

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import os

from datetime import datetime
from binance.client import Client
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor

In [2]:
# init
api_key = os.environ.get('BINANCE_API')
api_secret = os.environ.get('BINANCE_SECRET')

client = Client(api_key, api_secret)

print(client.get_account())
print(client.get_asset_balance(asset='BTC'))
print(client.get_all_tickers())

{'makerCommission': 10, 'takerCommission': 10, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': True, 'canDeposit': True, 'updateTime': 1618173055044, 'accountType': 'SPOT', 'balances': [], 'permissions': ['SPOT']}
None
[{'symbol': 'ETHBTC', 'price': '0.03923300'}, {'symbol': 'LTCBTC', 'price': '0.00512600'}, {'symbol': 'BNBBTC', 'price': '0.00855030'}, {'symbol': 'NEOBTC', 'price': '0.00154700'}, {'symbol': 'QTUMETH', 'price': '0.00750600'}, {'symbol': 'EOSETH', 'price': '0.00335500'}, {'symbol': 'SNTETH', 'price': '0.00009999'}, {'symbol': 'BNTETH', 'price': '0.00319400'}, {'symbol': 'BCCBTC', 'price': '0.07908100'}, {'symbol': 'GASBTC', 'price': '0.00028880'}, {'symbol': 'BNBETH', 'price': '0.21797700'}, {'symbol': 'BTCUSDT', 'price': '60404.71000000'}, {'symbol': 'ETHUSDT', 'price': '2369.72000000'}, {'symbol': 'HSRBTC', 'price': '0.00041400'}, {'symbol': 'OAXETH', 'price': '0.00017780'}, {'symbol': 'DNTETH', 'price': '0.00002801'}, {'symbol': 'MCOETH',

In [ ]:
btc_price = {'error':False}
def btc_trade_history(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        print(msg['c'])
        btc_price['last'] = msg['c']
        btc_price['bid'] = msg['b']
        btc_price['last'] = msg['a']
    else:
        btc_price['error'] = True

In [ ]:
# init and start the WebSocket
bsm = BinanceSocketManager(client)
conn_key = bsm.start_symbol_ticker_socket('BTCUSDT', btc_trade_history)
bsm.start()

In [ ]:
btc_price

In [ ]:
# stop websocket
bsm.stop_socket(conn_key)

# properly terminate WebSocket
reactor.stop()

In [4]:
[x for x in client.get_all_tickers() if x.get('symbol')[0:4] == 'DOGE']

[{'symbol': 'DOGEBNB', 'price': '0.00015270'},
 {'symbol': 'DOGEBTC', 'price': '0.00000455'},
 {'symbol': 'DOGEUSDT', 'price': '0.27412430'},
 {'symbol': 'DOGEPAX', 'price': '0.00216820'},
 {'symbol': 'DOGEUSDC', 'price': '0.00212770'},
 {'symbol': 'DOGEBUSD', 'price': '0.27491300'},
 {'symbol': 'DOGEEUR', 'price': '0.23098000'},
 {'symbol': 'DOGETRY', 'price': '2.23580000'},
 {'symbol': 'DOGEAUD', 'price': '0.36060000'},
 {'symbol': 'DOGEBRL', 'price': '1.57410000'},
 {'symbol': 'DOGEGBP', 'price': '0.20088000'}]

In [7]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
timestamp = client._get_earliest_valid_timestamp('DOGEUSDT', '1d')

# request historical candle (or klines) data
klines = client.get_historical_klines("DOGEUSDT", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

In [ ]:
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
timestamp

In [10]:
klines[-1]




[1618685700000,
 '0.27480000',
 '0.27489960',
 '0.27414070',
 '0.27449730',
 '832363.00000000',
 1618685759999,
 '228516.22384700',
 318,
 '363595.00000000',
 '99811.79834750',
 '0']

In [ ]:
from datetime import datetime
ts = int("1503014400000") / 1000

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))